In [ ]:
from datasets import load_dataset
data = load_dataset("bigbio/an_em",name="an_em_bigbio_kb",trust_remote_code=True)

In [ ]:
df = data["train"].to_pandas()

In [ ]:
try:    
    entities = df[df["entities"].apply(len) != 0]["entities"].tolist()
    entities = [item for sublist in entities for item in sublist]
    new_entites = {}
    for entity in entities: new_entites[entity["id"]] = f"{entity['type']}"
    for entity in set([new_entites[entity] for entity in new_entites.keys()]): print(entity)
except:
    pass

In [ ]:
try:
    relations = df[df["relations"].apply(len) != 0]["relations"].tolist()
    relations = [item for sublist in relations for item in sublist]
    for relation in set([f"{new_entites[relation['arg1_id']]} {relation['type'].upper()} {new_entites[relation['arg2_id']]}" for relation in relations]):
        print(relation)
except:
    pass

In [ ]:
from pathlib import Path
import os
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
from datasets import load_dataset
from huggingface_hub import list_datasets

In [ ]:
kb_data = []
for file in sorted(os.listdir(Path("../figures/data_card/KB"))):
    file = file.split("_")
    length = (len(file)//2) - 1
    kb_data.append("_".join(word for word in file[:length]))
kb_data = set(kb_data)

In [ ]:
def download_dataset_parallel(args)   : 
    def download_dataset(dataset):
        try:
            load_dataset(dataset,trust_remote_code=True)
            print(f"{dataset} SUCCESSfully cached!!!")
        except:
            print(f"{dataset} - ERROR!!!")
    ThreadPool(150).imap_unordered(download_dataset, args)

def bigbio_datasets():
    datasets = [dataset.id for dataset in list_datasets() if dataset.author == "bigbio"]
    return datasets

download_dataset_parallel(bigbio_datasets())